In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_parquet(output)

In [ ]:
tn_mas_cols = df.filter(like='tn_mas_1').columns
print(tn_mas_cols)

In [ ]:
df.periodo.max()

In [ ]:
# Convert categorical features to numerical format
for col in df.select_dtypes(include=['object']).columns:
    df[col] = pd.Categorical(df[col]).codes

In [ ]:
df = pd.read_parquet('/home/all4data118/buckets/b1/datasets/df_final_sellout_prod_client_train_fe_es_b.parquet')
tn_mas_cols = df.filter(like='tn_mas_1').columns
print(tn_mas_cols)

# train y test
# train y test
df=df[df.customer_id.isin([10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10011, 10012, 10013] )]
df=df[df.product_id.isin([20001, 20002, 20003, 20004, 20005, 20006, 20007, 20009, 20011, 20032] )]
df=df.reset_index(drop=True)
X_train=df[(df.periodo>=201801)&(df.periodo<=201908)]
X_test=df[(df.periodo==201909)]
Y_train=df[(df.periodo>=201801)&(df.periodo<=201908)][['id_row','tn_mas_2']]
Y_test=df[(df.periodo==201909)][['id_row','tn_mas_2']]

In [ ]:
X_train=X_train.drop(['id_row','tn_mas_2'],axis=1)
X_test=X_test.drop(['id_row','tn_mas_2'],axis=1)

Y_train_sc=Y_train[['tn_mas_2']]
Y_test_sc=Y_test[['tn_mas_2']]
# Ensure that the number of labels matches the number of data points
assert len(X_train) == len(Y_train_sc), "Mismatch in the number of labels for training set"
assert len(X_test) == len(Y_test_sc), "Mismatch in the number of labels for validation set"

In [ ]:
# Identify categorical features based on their data types
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()


In [ ]:
label_encoder = LabelEncoder()
# Create a dictionary to store the label encoders
label_encoders = {}

for feature in categorical_features:
    label_encoder = LabelEncoder()
    X_train[feature] = label_encoder.fit_transform(X_train[feature])
    X_test[feature] = label_encoder.transform(X_test[feature])
    
    # Save the label encoder to the dictionary
    label_encoders[feature] = label_encoder

# Save the label encoders to a file using joblibbuko118/exp/model_es_b/client_product
joblib.dump(label_encoders, '/home/all4data118/buckets/b1/exp/model_es_b/client_product/label_encoders.joblib')

In [ ]:
# para subir depsues
# Load the label encoders from the file
label_encoders = joblib.load('/home/all4data118/buckets/b1/exp/model_es_b/client_product/label_encoders.joblib')

# Assuming X_train and X_test are your datasets and categorical_features is the list of categorical feature names

# Apply the label encoders to the respective features
for feature, label_encoder in label_encoders.items():
    X_train[feature] = label_encoder.transform(X_train[feature])
    X_test[feature] = label_encoder.transform(X_test[feature])

In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming you have your data in X_train, Y_train_sc, X_test, and Y_test_sc

# Define your custom metric function (MAPE)
def mape(y_pred, dataset):
    y_true = dataset.get_label()
    absolute_percentage_errors = np.abs((y_true - y_pred) / y_true)
    return 'MAPE', np.mean(absolute_percentage_errors) * 100, False

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=Y_train['tn_mas_2_scaled'], categorical_feature=categorical_features)
val_data = lgb.Dataset(X_test, label=Y_test['tn_mas_2_scaled'], categorical_feature=categorical_features)

# Define LightGBM parameters
params = {
    'objective': 'regression',
  #  'metric': 'l1',  # Using 'l1' as the default metric for early stopping
   # 'boosting_type': 'gbdt',
   # 'num_leaves': 31,
   # 'learning_rate': 0.05,
   # 'feature_fraction': 0.9
}

# Train the LightGBM model with MAPE as the evaluation metric
model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, val_data],
    num_boost_round=1000,
    feval=mape
)


In [ ]:
# Obtain gain feature importance
gain_importance = model.feature_importance(importance_type='gain')

# Display feature importance with feature names
feature_names = model.feature_name()
gain_importance_df = pd.DataFrame({'Feature': feature_names, 'Gain': gain_importance})
gain_importance_df=gain_importance_df.sort_values(by='Gain', ascending=False)ç

# Obtain split feature importance
split_importance = model.feature_importance(importance_type='split')

# Display feature importance
feature_names = model.feature_name()
split_importance_df = pd.DataFrame({'Feature': feature_names, 'Split': split_importance})
split_importance_df=split_importance_df.sort_values(by='Split', ascending=False)

unique_features = list(set(split_importance_df.Feature[:500]).union(set(gain_importance_df.Feature[:500])))
unique_features

In [ ]:
X_train=X_train[unique_features]
X_test=X_test[unique_features]

In [ ]:
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import optuna

In [ ]:
import optuna
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import joblib
from tqdm import tqdm

# Assuming X_train, X_test, Y_train_sc, Y_test_sc are defined

# Define the number of trials
n_trials = 20

# Define a list of random seeds
seeds = [79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157,
         163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241,
         251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349,
         353, 359, 367, 373, 379, 383, 389, 397]

# Create a list to store the results
results = []

# Create a list to store successful trials
successful_trials = []

def objective(trial):
    # Select a random seed for this trial
    seed = np.random.choice(seeds)
    
    train_x, test_x, train_y, test_y = X_train, X_test, Y_train_sc, Y_test_sc
    param = {
        'metric': 'rmse', 
        'random_state': seed,  # Use the selected seed
        'n_estimators': 20000,
        'learning_rate': trial.suggest_categorical('learning_rate', [0.5, 0.4, 0.7, 1]),
        'max_depth': trial.suggest_categorical('max_depth', [3, 6, 9,12,16]),
        'num_leaves': trial.suggest_int('num_leaves', 8, 256),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300)
    }
    
    model = LGBMRegressor(**param, verbose=-1)  
    
    try:
        model.fit(train_x, train_y, eval_set=[(test_x, test_y)])  # Print every 500 iterations
        preds = model.predict(test_x)
        rmse = mean_squared_error(test_y, preds, squared=False)
        
        # Save the results for each trial
        results.append({'seed': seed, 'hyperparameters': param, 'rmse': rmse})
        
        # Save the model with a unique identifier
        model_name = f"/home/all4data118/buckets/b1/exp/model_es_b/client_product/model_trial_{trial.number}_seed_{seed}_rmse_{rmse:.4f}.joblib"
        joblib.dump(model, model_name)
        
        # Append trial parameters to successful_trials list
        successful_trials.append(param)
        
        return rmse
    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return float('inf')  # Return a large value to indicate failure
        
# Create a single study
study = optuna.create_study(direction='minimize')

# Optimize for the specified number of trials
study.optimize(objective, n_trials=n_trials)

# Print the best trial parameters
print('Best trial parameters:')
print(study.best_trial.params)

# Print the results for each trial
print('Results for each trial:')
for result in results:
    print(result)

# Print successful trial parameters
print('Successful trial parameters:')
for params in successful_trials:
    print(params)

In [ ]:
study.trials_dataframe().to_csv('/home/all4data118/buckets/b1/exp/model_es_b/client_product/trials_df.csv')

In [ ]:
pd.DataFrame(results).to_csv('/home/all4data118/buckets/b1/exp/model_es_b/client_product/results_df.csv')

In [ ]:
import joblib
# Create an empty DataFrame to store the inverse transformed values
inverse_transformed_df = pd.DataFrame()
grouped = y_test_df.groupby('product_id')['tn_mas_2_scaled']
column_to_standardize = 'tn_mas_2_scaled'


# Load the scaler object from the file
scalers = joblib.load('scalers_product_tn.joblib')

# Inverse transform each group and concatenate the results
for group_name, group_data in grouped:
    # Extract the values to be inverse transformed
    values_to_inverse_transform = y_test_df.loc[group_data.index][column_to_standardize].values.reshape(-1, 1)

    # Retrieve the scaler for this group from the dictionary
    scaler = scalers[group_name]

    # Inverse transform the data using the original mean and standard deviation
    inverse_transformed_values = scaler.inverse_transform(values_to_inverse_transform)

    # Create a DataFrame with the inverse transformed values and 'product_id' as the index
    group_df = pd.DataFrame(inverse_transformed_values, index=group_data.index, columns=[column_to_standardize])

    # Concatenate the group DataFrame to the overall inverse transformed DataFrame
    inverse_transformed_df = pd.concat([inverse_transformed_df, group_df])

# Sort the DataFrame by index to get the original order
y_test_df_transformed = inverse_transformed_df.sort_index()

In [ ]:
import joblib
# Create an empty DataFrame to store the inverse transformed values
inverse_transformed_df = pd.DataFrame()
grouped = y_pred_df.groupby('product_id')['y_pred']
column_to_standardize = 'y_pred'


# Load the scaler object from the file
scalers = joblib.load('scalers_product_tn.joblib')

# Inverse transform each group and concatenate the results
for group_name, group_data in grouped:
    # Extract the values to be inverse transformed
    values_to_inverse_transform = y_pred_df.loc[group_data.index][column_to_standardize].values.reshape(-1, 1)

    # Retrieve the scaler for this group from the dictionary
    scaler = scalers[group_name]

    # Inverse transform the data using the original mean and standard deviation
    inverse_transformed_values = scaler.inverse_transform(values_to_inverse_transform)

    # Create a DataFrame with the inverse transformed values and 'product_id' as the index
    group_df = pd.DataFrame(inverse_transformed_values, index=group_data.index, columns=[column_to_standardize])

    # Concatenate the group DataFrame to the overall inverse transformed DataFrame
    inverse_transformed_df = pd.concat([inverse_transformed_df, group_df])

# Sort the DataFrame by index to get the original order
y_pred_df_transformed = inverse_transformed_df.sort_index()


In [ ]:
y_test_df=pd.DataFrame({'product_id':df[(df.periodo==201902)]['product_id'].values,'tn_mas_2_scaled':df[(df.periodo==201902)]['tn_mas_2_scaled'].values,'tn_mas_2_real':df[(df.periodo==201902)]['tn_mas_2'].values})

In [ ]:
import joblib
# Create an empty DataFrame to store the inverse transformed values
inverse_transformed_df = pd.DataFrame()
grouped = y_test_df.groupby('product_id')['tn_mas_2_scaled']
column_to_standardize = 'tn_mas_2_scaled'


# Load the scaler object from the file
scalers = joblib.load('scalers_product_tn.joblib')

# Inverse transform each group and concatenate the results
for group_name, group_data in grouped:
    # Extract the values to be inverse transformed
    values_to_inverse_transform = y_test_df.loc[group_data.index][column_to_standardize].values.reshape(-1, 1)

    # Retrieve the scaler for this group from the dictionary
    scaler = scalers[group_name]

    # Inverse transform the data using the original mean and standard deviation
    inverse_transformed_values = scaler.inverse_transform(values_to_inverse_transform)

    # Create a DataFrame with the inverse transformed values and 'product_id' as the index
    group_df = pd.DataFrame(inverse_transformed_values, index=group_data.index, columns=[column_to_standardize])

    # Concatenate the group DataFrame to the overall inverse transformed DataFrame
    inverse_transformed_df = pd.concat([inverse_transformed_df, group_df])

# Sort the DataFrame by index to get the original order
y_test_df_transformed = inverse_transformed_df.sort_index()

In [ ]:
total=y_pred_df_transformed.join(y_test_df_transformed)

In [ ]:
total['diffe']=np.abs(total['y_pred']-total['tn_mas_2_scaled'])